In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"  # CSV DataFrames Random Statistics JWAS
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-07-23 05:16:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.7

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [ ]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Mixed Effects Neural Networks NNMM

# Part1. introduction

## 1. Overview
The Mixed Effect Neural Networks (NN-MM) extend linear mixed model ("MM") to multilayer neural networks ("NN") by adding one middle layer between genotype layer and phenotypes layer. Nodes in the middle layer represent intermediate traits, e.g., the known intermediate omics features such as gene expression levels can be incorporated in the middle layer. These three sequential layers form a unified network. 

![](https://github.com/zhaotianjing/figures/blob/main/omics_example.png?raw=true)

NN-MM allow any patterns of missing data in the middle layer, and missing data will be sampled. In below figure, for an individual, the gene expression levels of the first two genes are 0.9 and 0.1, respectively, and the gene expression level of the last gene is missing to be sampled. The missing patterns of gene expression levels can be different for different individuals.



## 2. Extend linear mixed model to multilayer neural networks

Multiple independent single-trait mixed models are used to model the relationships between input layer (genotypes) and middle layer (intermediate traits). Activation functions in the neural network are used to approximate the linear/nonlinear relationships between middle layer (intermediate traits) and output layer (phenotypes). Missing values in the middle layer (intermediate traits) are sampled by Hamiltonian Monte Carlo based on the upstream genotype layer and downstream phenotype layer.

Details can be found in our publications:

> * Tianjing Zhao, Jian Zeng, and Hao Cheng. Extend mixed models to multilayer neural networks for genomic prediction including intermediate omics data, GENETICS, 2022; [https://doi.org/10.1093/genetics/iyac034](https://doi.org/10.1093/genetics/iyac034). 
> * Tianjing Zhao, Rohan Fernando, and Hao Cheng. Interpretable artificial neural networks incorporating Bayesian alphabet models for genome-wide prediction and association studies, G3 Genes|Genomes|Genetics, 2021;  [https://doi.org/10.1093/g3journal/jkab228](https://doi.org/10.1093/g3journal/jkab228)

## 3. Flexibility

NN-MM can fit fully-connected neural networks ((a),(b)), or partial-connected neural networks ((c),(d)). Also, the relationship between middle layer (intermediate traits) and output layer (phenotypes) can be based on activation functions ((a),(c)), or pre-defined by a user-defined function ((b),(d)).

![](https://github.com/zhaotianjing/figures/blob/main/wiki_full_vs_partial.png?raw=true)

## 4. Multi-threaded parallelism

By default, multiple single-trait models will be used to model the relationships between input layer (genotypes) and middle layer (intermediate traits). Multi-threaded parallelism will be used for parallel computing. The number of threads can be checked by running `Threads.nthreads()` in Julia. Usually, using multiple threads will be about 3 times faster than using a single thread.

The number of execution threads is controlled by using the -t/--threads command-line argument (requires at least Julia 1.5). 

For example, to start Julia with 4 threads:

```
julia --threads 4
```

If you're using Juno via IDE like Atom, all threads will be loaded automatically. 

# Part 2: Mixed effect neural network: Genotypes -> Unobserved intemediate traits -> Phenotyes

Tips: please center the phenotypes to have zero mean.

## Example(a): fully-connected neural networks, all intemediate traits are unobserved
- nonlinear function (to define relationship between middle layer and phenotye): tanh (other supported activation functions: "sigmoid", "relu", "leakyrelu", "linear")
- number of nodes in the middle layer: 3
- Bayesian model: multiple independent single-trait BayesC (to sample marker effects on intemediate traits). Note, to use multi-trait Bayesian Alphabet models, please set `mega_trait=false` in `runMCMC()` function.
- sample the unobserved intemediate traits in the middle layer: Hamiltonian Monte Carlo

![](https://github.com/zhaotianjing/figures/blob/main/part2_example.png?raw=true)

In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,Random
Random.seed!(123)

# Step 2: Read data 
phenofile  = dataset("phenotypes.csv") #get example data path
genofile   = dataset("genotypes.csv")  #get example data path

phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"]) #read phenotypes (output layer)
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");                      #read genotypes  (input layer)


# Step 3: Build Model Equations 
model_equation  ="y1 = intercept + genotypes"  #name of phenotypes is "y1" in the phenotypes data
                                               #name of genotypes is "genotypes" (user-defined in the previous step)
                                               #the single-trait mixed model used between input and each node in middle layer is: middle node = intercept + genotypes
model = build_model(model_equation,
		    num_hidden_nodes=3,            #number of nodes in middle layer is 3
		    nonlinear_function="tanh");    #tanh function is used to approximate relationship between middle layer and phenotype


# Step 4: Run Analysis
out=runMCMC(model,phenotypes,chain_length=5000); 

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], phenotypes, on = :ID) 
accuruacy  = cor(results[!,:EBV],results[!,:bv1])

## Example output files

The i-th middle nodes will be named as "trait name"+"i". In our example, the observed trait is named "y1", and there are 3 middle nodes, so the middle nodes are named as "y11", "y12", and "y13", respectively.

Below is a list of files containing estimates and standard deviations for variables of interest. 

| file name      | description |
| -----------    | ----------- |
| EBV_NonLinear.txt | estimated breeding values for observed trait  |
| EBV_y11.txt       | estimated breeding values for middle node 1      |
| EBV_y12.txt       | estimated breeding values for middle node 2      |
| EBV_y13.txt       | estimated breeding values for middle node 3      |
|genetic_variance.txt                   | estimated genetic variance-covariance of all middle nodes |
|heritability.txt                       | estimated heritability of all middle nodes                |
|location_parameters.txt                | estimated bias of all middle nodes                        |
|neural_networks_bias_and_weights.txt.  | estimated bias of phenotypes and weights between middle nodes and phenotypes|
|pi_genotypes.txt                       | estimated pi of all middle nodes                          | 
|marker_effects_genotypes.txt           | estimated marker effects of all middle nodes              |
|residual_variance.txt                  | estimated residual variance-covariance for all middle nodes| 

Below is a list of files containing MCMC samples for variables of interest. 

| file name      | description |
| -----------    | ----------- |
| MCMC_samples_EBV_NonLinear.txt     | MCMC samples from the posterior distribution of breeding values for phenotypes  |
| MCMC_samples_EBV_y11.txt     | MCMC samples from the posterior distribution of breeding values for middle node 1       |
| MCMC_samples_EBV_y12.txt     | MCMC samples from the posterior distribution of breeding values for middle node 2       |
| MCMC_samples_EBV_y13.txt     | MCMC samples from the posterior distribution of breeding values for middle node 3       |
|MCMC_samples_genetic_variance.txt                   | MCMC samples from the posterior distribution of genetic variance-covariance for all middle nodes   |
|MCMC_samples_heritability.txt                       | MCMC samples from the posterior distribution of heritability for all middle nodes                                | 
|MCMC_samples_marker_effects_genotypes_y11            |MCMC samples from the posterior distribution of marker effects for middle node 1           |
|MCMC_samples_marker_effects_genotypes_y12            |MCMC samples from the posterior distribution of marker effects for middle node 2           |
|MCMC_samples_marker_effects_genotypes_y13            |MCMC samples from the posterior distribution of marker effects for middle node 3           |
|MCMC_samples_marker_effects_variances_genotypes.txt | MCMC samples from the posterior distribution of marker effect variance for all middle nodes        |
|MCMC_samples_neural_networks_bias_and_weights.txt.  | MCMC samples from the posterior distribution of bias of observed trait and weights between middle nodes and phenotypes|
|MCMC_samples_pi_genotypes.txt                       | MCMC samples from the posterior distribution of pi for all middle nodes                                          |
|MCMC_samples_residual_variance.txt                  |  MCMC samples from the posterior distribution of residual variance-covariance for all middle nodes |

# Part 3: Mixed effect neural network: Genotypes -> (complete/incomplete) Intemediate omics features -> Phenotyes

Tips:
* Put the names of omics features in the `build_model()` function through the `latent_traits` argument.
* If there are many omics features (e.g., 1000), it is recommanded to set `printout_model_info=false` in the `runMCMC()` function.
* Missing omics data for individuals in the training dataset (i.e., individuals with phenotypes) is allowed. When you read a file with missing values via the `CSV.read()` function, the `missingstrings` argument should be used to set sentinel values that will be parsed as `missing`.
* For individuals in the testing dataset (i.e., individuals without phenotypes), if the testing individuals have complete omics data, then incorporating the omics data of those individuals may improve the relationship between input layer (genotype) and middle layer (omics).

## example(o1): fully-connected neural networks with observed intemediate omics features

* nonlinear function (to define relationship between omics and phenotye): sigmoid (other supported activation functions: "tanh", "relu", "leakyrelu", "linear")
* number of omics features in the middle layer: 10
* Bayesian model: multiple independent single-trait BayesC (to sample marker effects on intemediate omics)
* sample the missing omics in the middle layer: Hamiltonian Monte Carlo

![](https://github.com/zhaotianjing/figures/blob/main/part3_example.png?raw=true)


In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets, Random, HTTP #HTTP to download demo data from github
Random.seed!(123)

# Step 2: Read data (from github)
phenofile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/y.csv").body
omicsfile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/omics.csv").body
genofile   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_n100_p200.csv").body
phenotypes = CSV.read(phenofile,DataFrame)
omics      = CSV.read(omicsfile,DataFrame)
geno_df    = CSV.read(genofile,DataFrame)

omics_names = names(omics)[2:end]  #get names of omics
insertcols!(omics,2,:y => phenotypes[:,:y], :bv => phenotypes[:,:bv]) #phenotype and omics should be in the same dataframe

genotypes = get_genotypes(geno_df,separator=',',method="BayesC")

# Step 3: Build Model Equations
model_equation  ="y = intercept + genotypes" #name of phenotypes is "y"
                                             #name of genotypes is "genotypes" (user-defined in the previous step)
                                             #the single-trait mixed model used between input and each omics is: omics = intercept + genotypes
model = build_model(model_equation,
		    num_hidden_nodes=10,          #number of omics in middle layer is 3
                    latent_traits=omics_names,    #name of all omics
		    nonlinear_function="sigmoid") #sigmoid function is used to approximate relationship between omics and phenotypes

# Step 4: Run Analysis
out=runMCMC(model, omics, chain_length=5000, printout_model_info=false);

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], omics, on = :ID)
accuruacy  = cor(results[!,:EBV],results[!,:bv])

## Includes a residual that is not mediated by other omics features

To include residuals polygenic component (i.e. directly from genotypes to phenotypes, not mediated by omics features), you can additional hidden nodes in the middle layer (see example (o2)). This can also be achieved in a partial-connected neural network in a same manner.

![](https://github.com/zhaotianjing/figures/blob/main/wiki_omics_residual.png?raw=true)

### example(o2): fully-connected neural network with residuals

For all individuals, this extra hidden node will be treated as unknown to be sampled.

In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets, Random, HTTP 
Random.seed!(123)

# Step 2: Read data (from github)
phenofile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/y.csv").body
omicsfile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/omics.csv").body
genofile   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_n100_p200.csv").body
phenotypes = CSV.read(phenofile,DataFrame)
omics      = CSV.read(omicsfile,DataFrame)
geno_df    = CSV.read(genofile,DataFrame)

insertcols!(omics, :residual => missing)  #create a hidden node to account for residuals
omics[!,:residual] = convert(Vector{Union{Missing,Float64}}, omics[!,:residual]) #transform the datatype is required for Julia
omics_names = names(omics)[2:end]  #get names of 10 omics and 1 hidden node
insertcols!(omics,2,:y => phenotypes[:,:y], :bv => phenotypes[:,:bv]) #phenotype and omics should be in the same dataframe

genotypes = get_genotypes(geno_df,separator=',',method="BayesC")

# Step 3: Build Model Equations
model_equation  ="y = intercept + genotypes" 
model = build_model(model_equation,
		    num_hidden_nodes=11,   #10 omcis and 1 hidden node
                    latent_traits=omics_names,
		    nonlinear_function="sigmoid")

# Step 4: Run Analysis
out = runMCMC(model,omics,chain_length=5000,printout_model_info=false)

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], omics, on = :ID)
accuruacy  = cor(results[!,:EBV],results[!,:bv])

Users can also add extra hidden nodes in the partial-connected neural network. Please check next documentation for building a partial-connected neural network.

## Output files

Same as those described in Part2.

### Julia Tips:

* You may want to set missing values manually, for example, setting the phenotypes for individuals in testing dataset as `missing`. Firstly, the type of  columns should be changed to allow `missing`, e.g., `phenotypes[!,:y] =  convert(Vector{Union{Missing,Float64}}, phenotypes[!,:y])`. Then, `missing` can be set manually, e.g., `phenotypes[10:11,:y1] .= missing` forces the 10th and 11th elements to be `missing`.

# Part 4: Partial-connected neural networks

In partial-connected neural networks, SNPs can be divided into groups by users, and each group connects to its own intermediate trait in the middle layer.  Genotype groups should be loaded seperatly.

## example(c): partial-connected neural networks, all intemediate traits are unobserved

- nonlinear function (to define relationship between middle layer and phenotye): tanh
- number of nodes in the middle layer: 3
- Bayesian model (to sample marker effects on intemediate traits): 
  - genotype group 1: single-trait BayesA
  - genotype group 2: single-trait BayesB
  - genotype group 3: single-trait BayesC
- sample the unobserved intemediate traits in the middle layer: Hamiltonian Monte Carlo

![](https://github.com/zhaotianjing/figures/blob/main/partial_example.png?raw=true)

In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,Random
Random.seed!(123)

# Step 2: Read data
phenofile   = dataset("phenotypes.csv")       
genofile1   = dataset("genotypes_group1.csv")  #path of genotype group1
genofile2   = dataset("genotypes_group2.csv")  #path of genotype group2
genofile3   = dataset("genotypes_group3.csv")  #path of genotype group3

phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
geno1  = get_genotypes(genofile1,separator=',',method="BayesA");   #read genotype group1
geno2  = get_genotypes(genofile2,separator=',',method="BayesB");   #read genotype group2
geno3  = get_genotypes(genofile3,separator=',',method="BayesC");   #read genotype group3

# Step 3: Build Model Equations
model_equation = "y1 = intercept + geno1 + geno2 + geno3";  #middle node1=intercept + geno1
                                                            #middle node2=intercept + geno2
                                                            #middle node3=intercept + geno3
model = build_model(model_equation,nonlinear_function="tanh")

# Step 4: Run Analysis
out = runMCMC(model, phenotypes, chain_length=5000);

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], phenotypes, on = :ID)
accuruacy  = cor(results[!,:EBV],results[!,:bv1])

## example(o3): partial-connected neural networks with intermediate omics data

* Same as for fully-connected neural network with intermediate omics data, the names of omics features should be put in the `build_model()` function through the `latent_traits` argument. The order of omics and the order of genotype groups in the model equation should be consistant.

In below example, we assume genotype group i only affect omics i (here i =1,...,5).

![](https://github.com/zhaotianjing/figures/blob/main/part4_partial_omics.png?raw=true)

In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,Random,HTTP 
Random.seed!(1)

# Step 2: Read data
phenofile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/y.csv").body
omicsfile  = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/omics.csv").body

genofile1   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_group1.csv").body
genofile2   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_group2.csv").body
genofile3   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_group3.csv").body
genofile4   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_group4.csv").body
genofile5   = HTTP.get("https://raw.githubusercontent.com/zhaotianjing/nnmm_doc/main/data_simulation/geno_group5.csv").body

phenotypes  = CSV.read(phenofile,DataFrame)
omics       = CSV.read(omicsfile,DataFrame)[:,1:6] #only use first 5 omics for demonstration
omics_names = names(omics)[2:end] #get names of omics
insertcols!(omics,2,:y => phenotypes[:,:y], :bv => phenotypes[:,:bv]) #phenotype and omics should be in the same dataframe

geno1_df    = CSV.read(genofile1,DataFrame)
geno2_df    = CSV.read(genofile2,DataFrame)
geno3_df    = CSV.read(genofile3,DataFrame)
geno4_df    = CSV.read(genofile4,DataFrame)
geno5_df    = CSV.read(genofile5,DataFrame)

geno1  = get_genotypes(geno1_df,separator=',',method="BayesA");
geno2  = get_genotypes(geno2_df,separator=',',method="BayesB");
geno3  = get_genotypes(geno3_df,separator=',',method="BayesC");
geno4  = get_genotypes(geno4_df,separator=',',method="RR-BLUP");
geno5  = get_genotypes(geno5_df,separator=',',method="BayesL");


# Step 3: Build Model Equations
model_equation = "y = intercept + geno1 + geno2 + geno3 + geno4 + geno5"; #omics1=intercept+geno1; omics2=intercept+geno2; ...
model = build_model(model_equation,
		    num_hidden_nodes=5,
		    nonlinear_function="sigmoid",
	            latent_traits=omics_names)

# Step 4: Run Analysis
out = runMCMC(model, omics, chain_length=5000, printout_model_info=false);

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], omics, on = :ID)
accuruacy  = cor(results[!,:EBV],results[!,:bv])

# Part 5: User-defined relationship between middle layer (intermediate traits) and output layer (phenotype)

* Firstly, a function should be pre-defined by user, where each function argument represents the input from a intermediate trait. For example, with two omics, we can have `pig_growth(omics1,omics2) = sqrt(omics1^2 / (omics1^2 + omics2^2))`)
* Then, put the name of the user-define function in the `nonlinear_function` arugument when build the model.

### example(b): user-defined relationship between middle layer (intermediate traits) and output layer (phenotype)

- number of nodes in the middle layer: 2
- nonlinear function (to define relationship between middle nodes and phenotype): y = sqrt(x1^2 / (x1^2 + x2^2))
- sample the missing omics in the middle layer: Matropolis-Hastings

All the other code are the same as before, except Step 3: Build Model Equations. Note that user-defined relationship is supported in both fully-connected and partial-connected neural network. 


```
# Step 3: Build Model Equations
pig_growth(x1,x2) = sqrt(x1^2 / (x1^2 + x2^2))
model = build_model(model_equation,
                    num_hidden_nodes=2,
                    nonlinear_function=pig_growth);
```